In [2]:
import pandas as pd
import numpy as np

In [14]:
#read in DNAclust file
with open("piRNAs_clust.csv") as c:
    lines = c.readlines()
    lines = "".join(lines)
    lines = lines.split("\n")
    clusters = [c.split("\t") for c in lines]
    clusters_filtered = []
    for c in clusters:
        del c[-1]        
        if 3 <= len(c):
            cluster=[]
            for ids in c:
                if ids == "":
                    pass
                else:
                    cluster.append(ids)
            clusters_filtered.append(cluster)
#read in novel_piRNA file
with open("multi_sample_piRNA.fasta") as unfiltered_fasta:
    lines = unfiltered_fasta.readlines()
    all_ids = lines[::2]
    reads = lines[1::2]
    seqs = pd.DataFrame({"IDs":all_ids, "Seqs": reads})
    seqs["IDs"] = seqs["IDs"].str.strip(">\n")
    seqs["Seqs"] = seqs["Seqs"].str.strip("\n")
    all_ids = list(seqs["IDs"])

In [4]:
samples = ["JEG", "JFC", "JMV", "KEG", "KFC", "KMV", "AEG", "AFC", "AMV"]
consensus = [i[0] for i in clusters_filtered]

In [5]:
consensus_seqs = seqs[seqs["IDs"].isin(consensus)]
consensus_seqs.index = consensus_seqs["IDs"]
consensus_seqs = consensus_seqs.drop(columns=["IDs"])

In [10]:
for s in samples:
    consensus_seqs[s] = 0
counter = 0
for cluster in clusters_filtered:
    counter+=1 
    for s in samples:
        count = 0
        for i, item in enumerate(cluster):
# So if sample name is in header split on x to extract the read count stored in the header
            if s in item:
                count += int(cluster[i].split("x")[1])
# Store the count per sample                
        consensus_seqs.loc[cluster[0],s] = count


In [11]:
# Creating identifier
consensus_seqs.index = [f"pred_piRNA_{i}" for i in range(0,consensus_seqs.shape[0])]
# Saving Dataframe
consensus_seqs.to_csv("counts_predicted_piRNAs.csv", sep = "\t")
display(consensus_seqs.head())

,Seqs,JEG,JFC,JMV,KEG,KFC,KMV,AEG,AFC,AMV
pred_piRNA_0,TAGCATTGGATCTTGACGCGAACAAATC,0,0,0,101,123,150,117,134,111
pred_piRNA_1,TTAGTTTGGGTAAATAATGATGAAAAT,0,0,0,124,150,0,147,152,0
pred_piRNA_2,TGTTATTATCAAAGACTTATCTGAGT,0,0,0,157,255,230,217,220,174
pred_piRNA_3,TCATCTAGCTCCATCGGTTCTGTAGCT,0,0,0,7710,8109,8828,7241,4879,8225
pred_piRNA_4,TTAAAAGAACGATAGATCTGCAACTTGC,0,0,0,101,0,135,151,109,170


In [15]:
# Writing all predicted (consensus) piRNAs to fasta 
with open("novel_pred_piRNAs.fasta","w") as f:
    for i in consensus_seqs.index:
        seq = consensus_seqs.loc[i,"Seqs"]
        f.write(f">{i}\n{seq}\n")